In [1]:
import itertools
import json
import numpy as np
from hmmlearn import hmm
from models_gaussian import StandardGaussianHMM, GaussianDenseHMM
import time
from tqdm import tqdm
import ssm
from ssm.util import find_permutation

np.random.seed(2022)

2022-07-27 10:27:11.104344: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-27 10:27:11.104425: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# ssm_hmm = ssm.HMM(n, 1, observations="categorical")
# ssm_hmm.observations.__init__(K=n, D=1, C=n_features)
# ssm_hmm.fit(X_true)
#
# most_likely_states = ssm_hmm.most_likely_states(X_true)
# ssm_hmm.permute(find_permutation(Y_true, most_likely_states))
# print(ssm_hmm.transitions.transition_matrix.round(2))
#
# (ssm_hmm.most_likely_states(X_true) == Y_true).mean()

In [2]:
def prepare_params(n, A_stat=False):
    pi = np.random.uniform(size=n)
    pi /= pi.sum()
    if A_stat:
        A = np.repeat(pi[np.newaxis,:], n, axis=0)
    else:
        A = np.exp(np.random.uniform(0, 5, size=(n, n)))
        A /= A.sum(axis=1)[:, np.newaxis]

    mu = np.random.randint(-10, 10, size=n)
    sigma = np.random.uniform(0.1,  1.5, size=n)
    return pi, A, mu, sigma


def my_hmm_sampler(pi, A,  mu, sigma, T):  # TODO
    n = pi.shape[0]
    X = [np.random.choice(np.arange(n), 1, replace=True, p=pi)]
    for t in range(T - 1):
        X.append(np.random.choice(np.arange(n), 1, replace=True, p=A[X[t][0], :]))
    Y = np.concatenate([np.random.normal(mu[s[0]], sigma[s[0]], 1) for s in X]).reshape(-1, 1)
    return X, Y


def experiment(n, m, T, s,  l, A_stat=False):
    pi, A, mu, sigma = prepare_params(n, A_stat)
    data = [my_hmm_sampler(pi, A, mu, sigma, T) for _ in range(s)]
    X_true = np.concatenate([x[1] for x in data])
    lenghts = [len(x[1]) for x in data]
    Y_true = np.concatenate([np.concatenate(y[0]) for y in data])
    print(X_true.shape)
    standard_acc = []
    standard_timer = []
    dense_acc = []
    dense_timer = []
    hmml_acc = []
    hmml_timer = []
    for _ in tqdm(range(3), desc="HMM"):
        # A_init = np.exp(np.random.uniform(0, 5, size=(n, n)))
        # A_init /= A_init.sum(axis=1)[:, np.newaxis]

        # start = time.perf_counter()
        # standardhmm = StandardGaussianHMM(n, em_iter=100, covariance_type='diag')
        # # standardhmm.transmat_ = A_init
        # standardhmm.fit(X_true, lenghts)
        # preds = np.concatenate([standardhmm.predict(x[1]) for x in data])
        # standard_timer.append(time.perf_counter() - start)
        # perm = find_permutation(preds, Y_true)
        # standard_acc.append((Y_true == np.array([perm[i] for i in preds])).mean())

        start = time.perf_counter()
        densehmm = GaussianDenseHMM(n, init_params="se",  mstep_config={"l_uz":  l, "l_vw": l}, covariance_type='full', em_iter=20)
        # densehmm.transmat_ = A_init
        densehmm.fit(X_true, lenghts)
        preds = np.concatenate([densehmm.predict(x[1]) for x in data])
        perm = find_permutation(preds, Y_true)
        dense_timer.append(time.perf_counter() - start)
        dense_acc.append((Y_true == np.array([perm[i] for i in preds])).mean())

        start = time.perf_counter()
        hmml = hmm.GaussianHMM(n, n_iter=100, init_params="s", covariance_type='full')
        # hmml.transmat_ = A_init
        hmml.fit(X_true, lenghts)
        preds = np.concatenate([hmml.predict(x[1]) for x in data])
        hmml_timer.append(time.perf_counter() - start)
        perm = find_permutation(preds, Y_true)
        hmml_acc.append((Y_true == np.array([perm[i] for i in preds])).mean())

    return {"standard_acc": standard_acc,
            "standard_time":  standard_timer,
            # "ssm_acc": ssm_acc,
            # "ssm_time":  ssm_time,
            "dense_acc": dense_acc,
            "dense_time": dense_timer,
            "hmml_acc": hmml_acc,
            "hmml_time": hmml_timer,
            "pi": pi,
            "A": A,
            "mu": mu,
            "sigma": sigma}


def run_experiments():
    results = []
    for  n, v, T, A_stat, l in itertools.product([2, 3, 4, 8],  [5, 10, 20], [10, 100, 1000],  [True, False],  [3,  4,  5]):
        tmp = experiment(n, v, T, 100, l, A_stat)
        results.append({**tmp, "n": n,  "v":  v, "T": T,  "A_stat": A_stat})
    return results

In [9]:
res_tmp = experiment(3,  5, 7, 8,  3, True)

(56, 1)


Fit model:   0%|                                                                                                                                                                    | 0/20 [00:00<?, ?it/s]

[[-1.87345582]
 [-1.3567729 ]
 [ 7.85496022]
 [-1.82839369]
 [ 7.02108315]
 [-1.23206356]
 [-0.86160177]]
(7, 1)
[[[22.01548722]]

 [[22.01548722]]

 [[22.01548722]]]
[[-7.33090193]
 [-1.0654756 ]
 [ 7.03192772]]
[[-0.98973823]
 [-0.79030296]
 [-0.52119322]
 [-1.26007697]
 [-2.13049329]
 [-0.91291899]
 [ 6.4045555 ]]
(7, 1)
[[[22.01548722]]

 [[22.01548722]]

 [[22.01548722]]]
[[-7.33090193]
 [-1.0654756 ]
 [ 7.03192772]]
[[-1.82750464]
 [-1.56041956]
 [-1.63605806]
 [-0.60528458]
 [-1.51682826]
 [-1.31446494]
 [-1.34209656]]
(7, 1)
[[[22.01548722]]

 [[22.01548722]]

 [[22.01548722]]]
[[-7.33090193]
 [-1.0654756 ]
 [ 7.03192772]]
[[-1.24947708]
 [ 5.00654833]
 [ 7.08869017]
 [-5.14216167]
 [-9.18718354]
 [ 6.57960094]
 [-6.79793569]]
(7, 1)
[[[22.01548722]]

 [[22.01548722]]

 [[22.01548722]]]
[[-7.33090193]
 [-1.0654756 ]
 [ 7.03192772]]
[[-7.24846813]
 [ 8.60814797]
 [-0.60699331]
 [ 7.29002797]
 [-1.95398524]
 [ 7.01100108]
 [ 7.17553396]]
(7, 1)
[[[22.01548722]]

 [[22.01548722]]


/ziob/klaudia/recommender_system/src/models/gaussian_dense_hmm/dense-hmm/code_dense_hmm/models_gaussian.py:892: RuntimeWarning: divide by zero encountered in true_divide
  _covars_ = ((covars_prior + c_n) /

Fit model:   5%|███████▊                                                                                                                                                    | 1/20 [00:00<00:16,  1.12it/s]

Loss at epoch 0 is 65.97370065
Loss at epoch 1 is 56.04595116
Loss at epoch 2 is 38.68493465
Loss at epoch 3 is 8.96047798
Loss at epoch 4 is -42.66569076
Loss at epoch 5 is -128.40850149
Loss at epoch 6 is -272.29458728
Loss at epoch 7 is -501.32942899
Loss at epoch 8 is -869.57866966
Loss at epoch 9 is -1556.43133381
[[-1.87345582]
 [-1.3567729 ]
 [ 7.85496022]
 [-1.82839369]
 [ 7.02108315]
 [-1.23206356]
 [-0.86160177]]
(7, 1)
[[[22.01548722]]

 [[22.01548722]]

 [[22.01548722]]]
[[0.]
 [0.]
 [0.]]
[[-0.98973823]
 [-0.79030296]
 [-0.52119322]
 [-1.26007697]
 [-2.13049329]
 [-0.91291899]
 [ 6.4045555 ]]
(7, 1)
[[[22.01548722]]

 [[22.01548722]]

 [[22.01548722]]]
[[0.]
 [0.]
 [0.]]
[[-1.82750464]
 [-1.56041956]
 [-1.63605806]
 [-0.60528458]
 [-1.51682826]
 [-1.31446494]
 [-1.34209656]]
(7, 1)
[[[22.01548722]]

 [[22.01548722]]

 [[22.01548722]]]
[[0.]
 [0.]
 [0.]]
[[-1.24947708]
 [ 5.00654833]
 [ 7.08869017]
 [-5.14216167]
 [-9.18718354]
 [ 6.57960094]
 [-6.79793569]]
(7, 1)
[[[22.01

...

/ziob/klaudia/recommender_system/src/models/gaussian_dense_hmm/dense-hmm/code_dense_hmm/models_gaussian.py:954: RuntimeWarning: divide by zero encountered in log
  log_A = np.log(self.transmat_)
/ziob/klaudia/recommender_system/src/models/gaussian_dense_hmm/dense-hmm/code_dense_hmm/models_gaussian.py:957: RuntimeWarning: divide by zero encountered in log
  log_pi = np.log(self.startprob_)

Fit model:  90%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌               | 18/20 [00:03<00:00,  7.63it/s]/ziob/klaudia/recommender_system/src/models/gaussian_dense_hmm/dense-hmm/code_dense_hmm/models_gaussian.py:954: RuntimeWarning: divide by zero encountered in log
  log_A = np.log(self.transmat_)
/ziob/klaudia/recommender_system/src/models/gaussian_dense_hmm/dense-hmm/code_dense_hmm/models_gaussian.py:957: RuntimeWarning: divide by zero encountered in log
  log_pi = np.log(self.startprob_)
/ziob/klaudi

[[-7.24846813]
 [ 8.60814797]
 [-0.60699331]
 [ 7.29002797]
 [-1.95398524]
 [ 7.01100108]
 [ 7.17553396]]
(7, 1)
[[[22.01548722]]

 [[22.01548722]]

 [[22.01548722]]]
[[0.]
 [0.]
 [0.]]
[[-1.09713147]
 [ 7.48857674]
 [-0.13672663]
 [-1.22131061]
 [ 0.25887364]
 [ 0.39231698]
 [ 6.64762793]]
(7, 1)
[[[22.01548722]]

 [[22.01548722]]

 [[22.01548722]]]
[[0.]
 [0.]
 [0.]]
[[-0.32663715]
 [ 5.51725098]
 [ 7.04671163]
 [-7.86424793]
 [ 7.76965919]
 [ 8.06176226]
 [ 6.97103327]]
(7, 1)
[[[22.01548722]]

 [[22.01548722]]

 [[22.01548722]]]
[[0.]
 [0.]
 [0.]]
[[-2.05603541]
 [-0.35187864]
 [-0.02752231]
 [-7.74541464]
 [-1.07828821]
 [-2.59434903]
 [ 0.44811754]]
(7, 1)
[[[22.01548722]]

 [[22.01548722]]

 [[22.01548722]]]
[[0.]
 [0.]
 [0.]]
(8, 7, 3)
(8, 7, 3)
[[0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]
[[0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]
[[0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0

/ziob/klaudia/recommender_system/src/models/gaussian_dense_hmm/dense-hmm/code_dense_hmm/models_gaussian.py:954: RuntimeWarning: divide by zero encountered in log
  log_A = np.log(self.transmat_)
/ziob/klaudia/recommender_system/src/models/gaussian_dense_hmm/dense-hmm/code_dense_hmm/models_gaussian.py:957: RuntimeWarning: divide by zero encountered in log
  log_pi = np.log(self.startprob_)
/ziob/klaudia/recommender_system/src/models/gaussian_dense_hmm/dense-hmm/code_dense_hmm/models_gaussian.py:892: RuntimeWarning: divide by zero encountered in true_divide
  _covars_ = ((covars_prior + c_n) /
/ziob/klaudia/recommender_system/src/models/gaussian_dense_hmm/dense-hmm/code_dense_hmm/models_gaussian.py:954: RuntimeWarning: divide by zero encountered in log
  log_A = np.log(self.transmat_)
/ziob/klaudia/recommender_system/src/models/gaussian_dense_hmm/dense-hmm/code_dense_hmm/models_gaussian.py:957: RuntimeWarning: divide by zero encountered in log
  log_pi = np.log(self.startprob_)

Fit mode

(8, 7, 3)
(8, 7, 3)
[[0.]
 [0.]
 [0.]]
Loss at epoch 0 is -20231889805451013548415468830798132692934127550896058655451723179872284360614506516110814306686220820105954406601520483969098472952083561442149335820357607359337474083492132099050394925050919722054999605248.00000000
Loss at epoch 1 is -825449901662167629143768283705928194739361450412616749676613103908780018424740083434284222637873236228823957138373406873932508372616362646685807900827932703669206751369499628572718312836387344988476499034112.00000000
Loss at epoch 2 is -33688470785202150554739925947755398225563700251779361265746284699588187209457744506823008172487745686198047281788897514995665294280431020550141056452420245402095729038959536901628170548203259478770657764835328.00000000
Loss at epoch 3 is -1374883054449030841796158737032227253821388145311890983852742787339103738369398300386986801504105334187687645152136839076762575488704622234831176179728700628799550164048792000461454326851371185830035059957039104.00000000
Loss at 

HMM: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:19<00:00,  6.36s/it]


In [10]:
# import tensorflow_probability as tfp
# tfd = tfp.distributions
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
# x = np.array([[1, .9], [.9,  -1]])
# tf.keras.constraints.non_neg().__call__((x + x.transpose())  / 2)

In [11]:
res_tmp

{'standard_acc': [],
 'standard_time': [],
 'dense_acc': [0.5892857142857143, 0.5892857142857143, 0.5],
 'dense_time': [6.151768825948238, 6.298039370100014, 6.1235468469094485],
 'hmml_acc': [1.0, 1.0, 1.0],
 'hmml_time': [0.183921309071593, 0.12134284200146794, 0.2036114139482379],
 'pi': array([0.30828399, 0.17292231, 0.51879371]),
 'A': array([[0.30828399, 0.17292231, 0.51879371],
        [0.30828399, 0.17292231, 0.51879371],
        [0.30828399, 0.17292231, 0.51879371]]),
 'mu': array([ 7, -8, -1]),
 'sigma': array([0.70871082, 1.41094953, 0.82751029])}

In [ ]:
n=10
A_stat=False
T=20
s=5400
l=5
pi, A, mu, sigma = prepare_params(n, A_stat)
data = [my_hmm_sampler(pi, A, mu, sigma, T) for _ in range(s)]
X_true = np.concatenate([x[1] for x in data])
lenghts = [len(x[1]) for x in data]
Y_true = np.concatenate([np.concatenate(y[0]) for y in data])
print(X_true.shape)
standard_acc = []
standard_timer = []
dense_acc = []
dense_timer = []
hmml_acc = []
hmml_timer = []
# for _ in tqdm(range(10), desc="HMM"):
    # A_init = np.exp(np.random.uniform(0, 5, size=(n, n)))
    # A_init /= A_init.sum(axis=1)[:, np.newaxis]

start = time.perf_counter()
standardhmm = StandardGaussianHMM(n, em_iter=100, covariance_type='full')
# standardhmm.transmat_ = A_init
standardhmm.fit(X_true, lenghts)
preds = np.concatenate([standardhmm.predict(x[1]) for x in data])
standard_timer.append(time.perf_counter() - start)
perm = find_permutation(preds, Y_true)
standard_acc.append((Y_true == np.array([perm[i] for i in preds])).mean())
#
start = time.perf_counter()
densehmm = GaussianDenseHMM(n, init_params="",  em_iter=100,  mstep_config={"l_uz":  l, "l_vw": l,  "em_lr": 0.001, "em_epochs":5})
# densehmm.transmat_ = A_init
densehmm.fit(X_true, lenghts)
preds = np.concatenate([densehmm.predict(x[1]) for x in data])
perm = find_permutation(preds, Y_true)
dense_timer.append(time.perf_counter() - start)
dense_acc.append((Y_true == np.array([perm[i] for i in preds])).mean())

start = time.perf_counter()
hmml = hmm.GaussianHMM(n, n_iter=100, init_params="stmc")
# hmml.transmat_ = A_init
hmml.fit(X_true, lenghts)
preds = np.concatenate([hmml.predict(x[1]) for x in data])
hmml_timer.append(time.perf_counter() - start)
perm = find_permutation(preds, Y_true)
hmml_acc.append((Y_true == np.array([perm[i] for i in preds])).mean())

(108000, 1)


Fit model:   0%|                                                                                                                                                                   | 0/100 [00:00<?, ?it/s]

In [ ]:
print(standard_acc)
print(standard_timer)

print(dense_acc)
print(dense_timer)

print(hmml_acc)
print(hmml_timer)

In [13]:
standardhmm = StandardGaussianHMM(n, em_iter=3, covariance_type='diag', params='mc')
standardhmm._init(X_true, lenghts)
standardhmm._compute_log_likelihood(X_true)
self = standardhmm
lengths = lenghts
val=None
val_lengths=None
X, n_seqs, max_seqlen = self._init(X_true, lengths=lenghts)
self._check()
stats = self._initialize_sufficient_statistics(n_seqs, max_seqlen)
log_dict = self._compute_metrics(X, lengths, stats, iter, 'i', val, val_lengths)
stats, total_logprob = self._forward_backward_gamma_pass(X, lengths, stats)
log_dict = self._compute_metrics(X, lengths, stats, iter, 'aE', val, val_lengths)
print("MEANS", self.means_)
self._do_mstep(stats)
# stats
# self._compute_metrics(X, lengths, stats, iter, 'aM', val, val_lengths)

print("MEANS", self.means_)

Even though the 'startprob_' attribute is set, it will be overwritten during initialization because 'init_params' contains 's'
Even though the 'transmat_' attribute is set, it will be overwritten during initialization because 'init_params' contains 't'
Even though the 'means_' attribute is set, it will be overwritten during initialization because 'init_params' contains 'm'


[[[11.112556]]

 [[11.112556]]

 [[11.112556]]]
(440, 1)
[[-7.61828361]
 [ 9.22388749]
 [ 7.94295517]
 [ 6.12625367]
 [ 8.42252181]
 [ 7.1734648 ]
 [ 9.23655742]
 [ 6.26868342]
 [ 7.27187161]
 [ 5.05003119]
 [ 6.30797773]
 [ 7.1990105 ]
 [ 5.21122106]
 [ 5.6726165 ]
 [ 8.77026933]
 [ 6.74420216]
 [ 7.78833227]
 [ 6.20614357]
 [-9.08504554]
 [ 5.76908056]
 [ 7.12923272]
 [ 6.96935421]
 [ 7.07361153]
 [ 6.38069951]
 [ 6.14916964]
 [ 7.88098842]
 [ 8.84812386]
 [ 5.88696747]
 [ 5.77515578]
 [ 7.50177838]
 [ 6.35604658]
 [ 8.74761786]
 [ 5.90448133]
 [ 7.2348336 ]
 [ 8.40076803]
 [ 5.50444951]
 [ 7.56499685]
 [ 6.48694921]
 [ 8.66309025]
 [ 6.10927846]
 [ 8.58007144]
 [ 5.65336615]
 [ 7.81673496]
 [ 5.57789865]]
(44, 1)
[[[11.112556]]

 [[11.112556]]

 [[11.112556]]]
[[ 8.09530471]
 [-8.23061461]
 [ 6.06922338]]
[[ 7.20182437]
 [ 5.53685572]
 [ 7.98807369]
 [ 8.61415712]
 [ 6.82361942]
 [ 9.10281846]
 [ 6.48590459]
 [-7.67485371]
 [ 7.98678878]
 [ 6.75867778]
 [ 6.3001999 ]
 [ 6.3166031 ]


/ziob/klaudia/miniconda3/envs/hmm/lib/python3.10/site-packages/hmmlearn/hmm.py:272: RuntimeWarning: invalid value encountered in true_divide
  self.means_ = ((means_weight * means_prior + stats['obs'])
